In [2]:
import pandas as pd

In [8]:

# 1) CSV 또는 DataFrame 불러오기
df = pd.read_csv("./01_raw_data/기온, 강수량.csv",encoding='cp949')  # 또는 직접 DataFrame 구성

# 2) 지점명 → 시도명 매핑
station_to_province = {
    '속초': '강원', '북춘천': '강원', '철원': '강원', '동두천': '경기', '파주': '경기',
    '대관령': '강원', '춘천': '강원', '백령도': '인천', '북강릉': '강원', '강릉': '강원',
    '동해': '강원', '서울': '서울', '인천': '인천', '원주': '강원', '울릉도': '경북',
    '수원': '경기', '영월': '강원', '충주': '충북', '서산': '충남', '울진': '경북',
    '청주': '충북', '대전': '대전', '추풍령': '충북', '안동': '경북', '상주': '경북',
    '포항': '경북', '군산': '전북', '대구': '대구', '전주': '전북', '울산': '울산',
    '창원': '경남', '광주': '광주', '부산': '부산', '통영': '경남', '목포': '전남',
    '여수': '전남', '흑산도': '전남', '완도': '전남', '고창': '전북', '순천': '전남',
    '홍성': '충남', '제주': '제주', '고산': '제주', '성산': '제주', '서귀포': '제주',
    '진주': '경남', '강화': '인천', '양평': '경기', '이천': '경기', '인제': '강원',
    '홍천': '강원', '태백': '강원', '정선군': '강원', '제천': '충북', '보은': '충북',
    '천안': '충남', '보령': '충남', '부여': '충남', '금산': '충남', '세종': '세종',
    '부안': '전북', '임실': '전북', '정읍': '전북', '남원': '전북', '장수': '전북',
    '고창군': '전북', '영광군': '전남', '김해시': '경남', '순창군': '전북', '북창원': '경남',
    '양산시': '경남', '보성군': '전남', '강진군': '전남', '장흥': '전남', '해남': '전남',
    '고흥': '전남', '의령군': '경남', '함양군': '경남', '광양시': '전남', '진도군': '전남',
    '봉화': '경북', '영주': '경북', '문경': '경북', '청송군': '경북', '영덕': '경북',
    '의성': '경북', '구미': '경북', '영천': '경북', '경주시': '경북', '거창': '경남',
    '합천': '경남', '밀양': '경남', '산청': '경남', '거제': '경남', '남해': '경남'
}

# 3) 시도명 추가
df['시도'] = df['지점명'].map(station_to_province)

# 4) 수치형 컬럼 변환 (문자 → 숫자, 에러는 NaN)
cols_to_numeric = ['평균기온(°C)', '평균최고기온(°C)', '평균최저기온(°C)', '월합강수량(00~24h만)(mm)']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 5) 그룹별 평균 계산: [일시, 시도] 기준
grouped = (
    df
    .dropna(subset=['시도'])  # 시도 매핑되지 않은 행 제거
    .groupby(['일시', '시도'])[cols_to_numeric]
    .mean()
    .reset_index()
)
# 1) 전국 평균 계산
national = (
    grouped
    .groupby('일시')[cols_to_numeric]   # 일시별
    .mean()                             # 수치형 컬럼 평균
    .reset_index()
)
national['시도'] = '전국'                # 시도명으로 “전국” 추가
# 컬럼 순서 맞추기
national = national[['일시', '시도'] + cols_to_numeric]

# 2) province별 + 전국 합치기
grouped_all = pd.concat([grouped, national], ignore_index=True)
# 6) 결과 확인


grouped_all.to_csv("./02_interim_data/temperature_rain_data.csv", index=False, encoding='utf-8-sig')
grouped_all.tail(50)


,일시,시도,평균기온(°C),평균최고기온(°C),평균최저기온(°C),월합강수량(00~24h만)(mm)
1030,2020-11,전국,9.211235,14.496096,4.534959,43.995372
1031,2020-12,전국,1.241363,6.416051,-3.111724,10.113855
1032,2021-01,전국,-0.222713,4.895683,-5.025523,26.400749
1033,2021-02,전국,4.131879,9.788018,-1.040033,22.521179
1034,2021-03,전국,9.284423,15.108142,4.103751,110.800303
1035,2021-04,전국,13.705214,19.514444,8.305543,75.530279
1036,2021-05,전국,17.058536,22.358030,12.198018,140.826555
1037,2021-06,전국,22.127857,26.947899,18.141064,103.364220
1038,2021-07,전국,26.474134,30.941960,22.909504,209.755835
1039,2021-08,전국,25.246042,29.446589,22.113454,292.891699
